In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_excel
import numpy as np
from pandas import DataFrame
from pandas import concat
from matplotlib.font_manager import FontProperties  # 字体管理器

import pandas as pd
import jieba
from snownlp import SnowNLP


from numpy import concatenate
from math import sqrt
from matplotlib.font_manager import FontProperties  # 字体管理器
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'




In [2]:
# 设置汉字格式
font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=15)
def str_to_float(s):
    s=s[:-1]
    s_float=float(s)
    return s_float

# 导入并处理数据

In [3]:
file_path = 'News Headlines.xlsx'

df = pd.read_excel(file_path)
df.columns = df.iloc[0]
df = df.drop(index=0)


In [4]:
file1 = 'data_RF.xlsx'
file2 = 'data_CountryGarden.xlsx' 
file3 = 'data_Evergrande.xlsx'
file4 = 'data_GiantLeap.xlsx'

df_RF = pd.read_excel(file1)
df_CountryGarden = pd.read_excel(file2)  
df_Evergrande = pd.read_excel(file3)
df_GiantLeap = pd.read_excel(file4)

In [5]:
df_RF['Date'] = pd.to_datetime(df_RF['Spread Date']).dt.date 
df_CountryGarden['Date'] = pd.to_datetime(df_CountryGarden['Spread Date']).dt.date 
df_Evergrande['Date'] = pd.to_datetime(df_Evergrande['Spread Date']).dt.date 
df_GiantLeap['Date'] = pd.to_datetime(df_GiantLeap['Spread Date']).dt.date 

df_RF = df_RF.drop(columns='Spread Date') 
df_CountryGarden = df_CountryGarden.drop(columns='Spread Date') 
df_GiantLeap = df_GiantLeap.drop(columns='Spread Date') 
df_Evergrande= df_Evergrande.drop(columns='Spread Date') 

df_RF = df_RF.set_index('Date') 
df_CountryGarden = df_CountryGarden.set_index('Date')  
df_GiantLeap = df_GiantLeap.set_index('Date')  
df_Evergrande = df_Evergrande.set_index('Date')  



In [6]:
df_RF

,Credit Spread,Excess Spread,CFETS Valuation
Date,,,
2019-06-18,214.360,126.230,5.27300
2019-06-19,216.375,126.760,5.29695
2019-06-20,218.975,128.990,5.26535
2019-06-21,216.930,128.095,5.27720
2019-06-24,215.075,128.400,5.27335
...,...,...,...
2023-08-24,5640.650,NaN,58.72440
2023-08-25,5645.780,NaN,58.79600
2023-08-28,5665.500,NaN,59.01210


# 计算sentiment value

In [7]:
titles = df['标题']
sentiments = []
for title in titles:
    s = SnowNLP(title)  
    sentiments.append(s.sentiments)

df['Sentiment Value'] = sentiments

In [8]:
# 转换时间格式  
df['Date'] = pd.to_datetime(df['新闻发布时间']).dt.date 

# 筛选出恒大的相关

In [9]:
# 筛选包含"恒大"的行
df_news_Evergrande = df[df['关联公司'].str.contains('恒大')]

# 对日期进行分组,sentiment取平均   
df_news_Evergrande= df_news_Evergrande.groupby('Date', as_index=False)['Sentiment Value'].mean()

df_news_Evergrande= df_news_Evergrande.set_index('Date')

In [10]:
# 筛选包含"富力"的行
df_news_RF= df[df['关联公司'].str.contains('富力')]

# 对日期进行分组,sentiment取平均   
df_news_RF = df_news_RF.groupby('Date', as_index=False)['Sentiment Value'].mean()
df_news_RF = df_news_RF.set_index('Date')

In [11]:
# 筛选包含“碧桂园”或"腾越"的行
df_news_CountryGarden= df[df['关联公司'].str.contains('碧桂园'or '腾越')]

# 对日期进行分组,sentiment取平均   
df_news_CountryGarden = df_news_CountryGarden.groupby('Date', as_index=False)['Sentiment Value'].mean()
df_news_CountryGarden = df_news_CountryGarden.set_index('Date')

# 变更表格格式为“日期”及“sentiment”两列

In [12]:
df_news_Evergrande

,Sentiment Value
Date,
2018-10-01,0.960081
2018-10-05,0.962679
2018-10-08,0.020003
2018-10-12,0.912739
2018-10-23,0.987970
...,...
2023-08-26,0.683534
2023-08-27,0.477571
2023-08-28,0.599453


In [13]:
df_news_RF

,Sentiment Value
Date,
2018-09-17,0.974873
2018-09-18,0.786056
2018-09-28,0.468201
2018-10-24,0.289513
2018-11-05,0.371126
...,...
2023-08-24,0.012630
2023-08-25,0.023769
2023-08-28,0.195333


In [14]:
df_news_CountryGarden

,Sentiment Value
Date,
2018-10-05,0.962679
2019-01-11,0.801190
2019-01-16,0.452439
2019-02-02,0.963229
2019-02-20,0.586033
...,...
2023-08-25,0.550040
2023-08-26,0.916423
2023-08-28,0.761335


# 合并表格

In [15]:
df_RF_final = df_RF.join(df_news_RF, how='outer') 
df_RF_final = df_RF_final[df_RF_final['Credit Spread'].notna()]
df_RF_final['Sentiment Value'] = df_RF_final['Sentiment Value'].fillna(0.5)
nan_rows = df_RF_final[df_RF_final['Excess Spread'].isna()]
df_RF_final = df_RF_final.dropna(subset=['Excess Spread'])
df_RF_final.to_csv('df_RF_final.csv', index=True)

In [16]:
df_CountryGarden_final = df_CountryGarden.join(df_news_CountryGarden, how='outer') 
df_CountryGarden_final = df_CountryGarden_final[df_CountryGarden_final['Credit Spread'].notna()]
df_CountryGarden_final['Sentiment Value'] = df_CountryGarden_final['Sentiment Value'].fillna(0.5)
df_CountryGarden_final.to_csv('df_CountryGarden_final.csv', index=True)

In [17]:
df_Evergrande_final = df_Evergrande.join(df_news_Evergrande, how='outer') 
df_Evergrande_final = df_Evergrande_final[df_Evergrande_final['Credit Spread'].notna()]
df_Evergrande_final['Sentiment Value'] = df_Evergrande_final['Sentiment Value'].fillna(0.5)
nan_rows = df_Evergrande_final[df_Evergrande_final['Excess Spread'].isna()]
df_Evergrande_final = df_Evergrande_final.dropna(subset=['Excess Spread'])
df_Evergrande_final.to_csv('df_Evergrande_final.csv', index=True)

In [18]:
df_GiantLeap_final = df_GiantLeap.join(df_news_CountryGarden, how='outer') 
df_GiantLeap_final = df_GiantLeap_final[df_GiantLeap_final['Credit Spread'].notna()]
df_GiantLeap_final['Sentiment Value'] = df_GiantLeap_final['Sentiment Value'].fillna(0.5)
df_GiantLeap_final.to_csv('df_GiantLeap_final.csv', index=True)

In [19]:
df_CountryGarden_final


,Credit Spread,Excess Spread,CFETS Valuation,Sentiment Value
Date,,,,
2019-06-18,226.135,142.280,5.43405,0.500000
2019-06-19,229.885,142.345,5.45830,0.500000
2019-06-20,229.630,142.665,5.43050,0.500000
2019-06-21,230.595,144.110,5.43590,0.500000
2019-06-24,228.630,144.350,5.43180,0.500000
...,...,...,...,...
2023-08-24,163.845,99.790,3.89305,0.656786
2023-08-25,162.460,99.520,3.89900,0.550040
2023-08-28,160.920,99.340,3.90300,0.761335


In [20]:
df_GiantLeap_final


,Credit Spread,Excess Spread,CFETS Valuation,Sentiment Value
Date,,,,
2019-06-18,252.94,166.485,5.5870,0.500000
2019-06-19,255.36,166.890,5.4991,0.500000
2019-06-20,259.85,169.600,5.5165,0.500000
2019-06-21,254.37,165.320,5.4468,0.500000
2019-06-24,252.45,165.510,5.4397,0.500000
...,...,...,...,...
2023-04-17,2012.42,1251.660,22.3996,0.525010
2023-04-18,2013.79,1250.980,22.4021,0.999738
2023-04-19,2012.29,1250.400,22.3856,0.711661


In [21]:
df_Evergrande_final


,Credit Spread,Excess Spread,CFETS Valuation,Sentiment Value
Date,,,,
2019-06-18,278.930,179.735,6.12475,0.500000
2019-06-19,274.270,173.780,5.99855,0.500000
2019-06-20,275.110,173.095,5.96650,0.225833
2019-06-21,274.775,174.000,5.97785,0.270513
2019-06-24,272.570,174.600,5.97385,0.500000
...,...,...,...,...
2021-07-12,2663.130,1719.130,29.18330,0.500000
2021-07-13,2694.600,1752.190,29.48310,0.982656
2021-07-14,3034.400,2092.190,32.85240,0.500000


In [22]:
df_RF_final


,Credit Spread,Excess Spread,CFETS Valuation,Sentiment Value
Date,,,,
2019-06-18,214.360,126.230,5.27300,0.500000
2019-06-19,216.375,126.760,5.29695,0.500000
2019-06-20,218.975,128.990,5.26535,0.786904
2019-06-21,216.930,128.095,5.27720,0.991536
2019-06-24,215.075,128.400,5.27335,0.500000
...,...,...,...,...
2021-12-08,7981.660,7019.750,82.20530,0.402639
2021-12-09,7980.770,7018.770,82.20550,0.197239
2021-12-10,8686.290,7724.490,89.24160,0.500000
